In [1]:
!pip install langchain-text-splitters langchain pymupdf langchain_ollama psycopg2-binary

  Using cached langchain_text_splitters-0.3.6-py3-none-any.whl.metadata (1.9 kB)
  Using cached langchain-0.3.19-py3-none-any.whl.metadata (7.9 kB)
  Using cached pymupdf-1.25.3-cp39-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (3.4 kB)
  Using cached langchain_ollama-0.2.3-py3-none-any.whl.metadata (1.9 kB)
  Using cached psycopg2_binary-2.9.10-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.9 kB)
  Using cached langchain_core-0.3.39-py3-none-any.whl.metadata (5.9 kB)
  Using cached langsmith-0.3.11-py3-none-any.whl.metadata (14 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached aiohttp-3.11.13-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached ollama-0.4.7-py3-none-any.whl.metadata (4.7 kB)
  Using cached a

In [ ]:
import urllib.request
import hashlib
from io import BytesIO

import pymupdf  # PyMuPDF
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=50,  
    separators=["\n\n", "\n", " ", ""],  # Prioritize logical breaks
)

from langchain_ollama import OllamaEmbeddings
embed = OllamaEmbeddings(
    model="all-minilm",
    base_url = "http://host.docker.internal:11434"
)
pg_pool = pool.SimpleConnectionPool(
    minconn=cores,  # Minimum connections
    maxconn=cores, # Maximum connections
    dsn=f"dbname={db} user={user} password={password} host={host}"
)

# Load the PDF
url = "https://arxiv.org/pdf/2302.12854"  # PDF path or URL

stream = BytesIO(urllib.request.urlopen(url).read())
doc = pymupdf.open(stream=stream, filetype="pdf")
for page in doc:
    chunks = text_splitter.split_text(page.get_text())
    embeddings = embed.embed_documents(chunks)
    print(len(chunks))
    print(len(embeddings))
    for chunk_id, (chunk, embedding) in enumerate(zip(chunks,embeddings)):
        print(f"Page: {page.number}, Chunk: {chunk_id}, Content: {chunk}, Embedding: {embedding}")
        print("---")

In [7]:
from queue import Queue
import multiprocessing
from threading import Thread
from psycopg2 import pool
import time
import hashlib
from io import BytesIO
import io
import random
import requests
import pymupdf  # PyMuPDF
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings

host = "pdf-service"
user = "postgres"
password = "admin"
db = "postgres"
#urls = ["https://arxiv.org/pdf/2302.12854"] # PDF path or URL
urls = ["https://arxiv.org/pdf/2502.14927", "https://arxiv.org/pdf/2502.14980", "https://arxiv.org/pdf/2502.15150", "https://arxiv.org/pdf/2502.15268", "https://arxiv.org/pdf/2502.15272", "https://arxiv.org/pdf/2502.15337", "https://arxiv.org/pdf/2502.15388", "https://arxiv.org/pdf/2502.15460", "https://arxiv.org/pdf/2502.15510", "https://arxiv.org/pdf/2502.15658", "https://arxiv.org/pdf/2502.14953", "https://arxiv.org/pdf/2502.14955", "https://arxiv.org/pdf/2502.14962", "https://arxiv.org/pdf/2502.15108", "https://arxiv.org/pdf/2502.15279", "https://arxiv.org/pdf/2502.15541", "https://arxiv.org/pdf/2502.15593", "https://arxiv.org/pdf/2502.15627", "https://arxiv.org/pdf/2404.15977", "https://arxiv.org/pdf/2409.09802", "https://arxiv.org/pdf/2409.16328", "https://arxiv.org/pdf/2410.00264", "https://arxiv.org/pdf/2412.18651", "https://arxiv.org/pdf/2412.20983", "https://arxiv.org/pdf/2501.18144", "https://arxiv.org/pdf/2502.05644", "https://arxiv.org/pdf/2502.09210", "https://arxiv.org/pdf/2301.04677", "https://arxiv.org/pdf/2305.00222", "https://arxiv.org/pdf/2308.16076", "https://arxiv.org/pdf/2403.04364", "https://arxiv.org/pdf/2403.13484", "https://arxiv.org/pdf/2403.18772", "https://arxiv.org/pdf/2403.19520", "https://arxiv.org/pdf/2405.01887", "https://arxiv.org/pdf/2406.00207", "https://arxiv.org/pdf/2412.17898", "https://arxiv.org/pdf/2501.18730"]

sources_insert = "INSERT INTO sources (id, uri) VALUES (%s, %s)"
#"""INSERT INTO sources (id, uri, title, summary, title_embedding, summary_embedding)
#VALUES (%s, %s, %s, %s, %s, %s)"""
semantic_insert = "INSERT INTO semantic_search (id, page, chunk, embedding) VALUES (%s, %s, %s, %s)"
keyword_insert = "INSERT INTO keyword_search (id, page, ts) VALUES (%s, %s, to_tsvector('english', %s))"

def thread_task(inq,con,core,chunk_size=500, chunk_overlap=50):
    embed = OllamaEmbeddings(
        model="all-minilm",
        base_url = "http://host.docker.internal:11434"
    )
    # LangChain's Recursive Text Splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, 
        chunk_overlap=chunk_overlap,  
        separators=["\n\n", "\n", " ", ""],  # Prioritize logical breaks
    )
    while True:
        with con.cursor() as cur:
            time.sleep(random.random()*(cores/2))
            url = inq.get()
            if url is None:
                inq.task_done()
                con_queue.put(con)
                break
            print(f"Processing PDF: {url}")
            try:
                key = hashlib.md5(url.encode()).hexdigest()
                with BytesIO() as stream_buffer:
                    response = requests.get(url, stream=True)
                    if response.status_code != 200:
                        raise Exception(f"Failed to download PDF; received HTTP {response.status_code} from underlying server")
                    for chunk in response.iter_content(chunk_size=io.DEFAULT_BUFFER_SIZE):
                        stream_buffer.write(chunk)
                    doc = pymupdf.open(stream=stream_buffer, filetype="pdf")
                    cur.execute(sources_insert,(key,url,))
                    for page in doc:
                        try:
                            cur.execute(keyword_insert,(key,page.number,page.get_text().replace('\x00', ''),))
                            chunks = text_splitter.split_text(page.get_text())
                            embeddings = embed.embed_documents(chunks)
                            for chunk_id, (chunk,embedding) in enumerate(zip(chunks,embeddings)):
                                cur.execute(semantic_insert,(key,page.number,chunk_id,embedding,))
                        except Exception as e:
                            print(f"Error processing page {page.number} of {url}: {e}")
                            continue  # Skip to the next page
            except Exception as e:
                print(f"Error downloading PDF from url: {url}: {e}")
                continue  # Skip to the next page

            con.commit()
            inq.task_done()

cores = multiprocessing.cpu_count()
insert_queue = Queue()

for url in urls:
    insert_queue.put(url)
# Create a connection pool
pg_pool = pool.SimpleConnectionPool(
    minconn=cores,  # Minimum connections
    maxconn=cores, # Maximum connections
    dsn=f"dbname={db} user={user} password={password} host={host}"
)
con_queue = Queue()
for core in range(0,cores):
    insert_queue.put(None)
    con_queue.put(None)

print("Starting insert of documents into Postgres")
insert_start_time = time.time()
join_list = []
for core in range(0,cores):
    worker = Thread(target=thread_task, args=(insert_queue,pg_pool.getconn(),core))
    worker.start()
    join_list.append(worker)
        
for thread in join_list:
    thread.join()

while not con_queue.empty():
    connection = con_queue.get()
    if connection is None:
        con_queue.task_done()
        break
    connection.close()
    con_queue.task_done()

insert_end_time = time.time()
print(f"Completed inserting documents in {insert_end_time - insert_start_time} seconds")

Starting insert of documents into Postgres
Processing PDF: https://arxiv.org/pdf/2502.14927
Processing PDF: https://arxiv.org/pdf/2502.14980
Processing PDF: https://arxiv.org/pdf/2502.15150
Error downloading PDF from url: https://arxiv.org/pdf/2502.14980: ('Connection broken: IncompleteRead(2097152 bytes read, 1253046 more expected)', IncompleteRead(2097152 bytes read, 1253046 more expected))
Processing PDF: https://arxiv.org/pdf/2502.15268
Processing PDF: https://arxiv.org/pdf/2502.15272
Processing PDF: https://arxiv.org/pdf/2502.15337
Processing PDF: https://arxiv.org/pdf/2502.15388
Processing PDF: https://arxiv.org/pdf/2502.15460
Processing PDF: https://arxiv.org/pdf/2502.15510
Processing PDF: https://arxiv.org/pdf/2502.15658
Processing PDF: https://arxiv.org/pdf/2502.14953
Processing PDF: https://arxiv.org/pdf/2502.14955
Processing PDF: https://arxiv.org/pdf/2502.14962
Processing PDF: https://arxiv.org/pdf/2502.15108
Processing PDF: https://arxiv.org/pdf/2502.15279
Processing PDF: 

In [ ]:
import psycopg2

conn = psycopg2.connect("dbname=mydb user=myuser password=mypass host=localhost")
cur = conn.cursor()

cur.execute("""
    INSERT INTO sources (id, uri, title, summary, title_embedding, summary_embedding)
    VALUES (%s, %s, %s, %s, %s, %s)
""", (doc_id, "path/to/file.pdf", title, summary, title_embedding, summary_embedding))

conn.commit()

In [5]:
# for page_num, page in enumerate(doc.pages):
#     chunks = page.chunk()  # Split into smaller parts

#     for chunk_idx, chunk in enumerate(chunks):
#         embedding = doc.embed(chunk.text)

#         # Insert into semantic_search
#         cur.execute("""
#             INSERT INTO semantic_search (id, page, chunk, embedding)
#             VALUES (%s, %s, %s, %s)
#         """, (doc_id, page_num + 1, chunk_idx, embedding))

#         # Insert into keyword_search
#         cur.execute("""
#             INSERT INTO keyword_search (id, page, ts)
#             VALUES (%s, %s, to_tsvector('english', %s))
#         """, (doc_id, page_num + 1, chunk.text))

# conn.commit()
# cur.close()
# conn.close()

Token indices sequence length is longer than the specified maximum sequence length for this model (525 > 512). Running this sequence through the model will result in indexing errors


text="FRANK ELAVSKY, Carnegie Mellon University, fje@cmu.edu\nAcademic, peer-reviewed 'short' papers are a common way to present a late-breaking work to the academic community that outlines preliminary findings, research ideas, and novel conversations. By comparison, blogging or writing posts on social media are an unstructured and open way to discuss ideas and start new conversations. Both have limitations in the proliferation of research ideas. The short paper format relies on the conference and journal submission process while blogging does not operate within a structured format or set of expectations at all. However, at times the demand exists for late-breaking ideas and conversations to arise in a raw form or with urgency but should still be archived and recorded in a way that promotes citational honesty and integrity. To address this, I present: The Micro-Paper, as a micro-paper itself. The Micro-Paper is a small, cheap, accessible, digital document that is self-published and arc